In [ ]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'redpanda-1:9092'
server = 'localhost:9092'


producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

# PRODUCE DATA

In [ ]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'redpanda-1:9092'
server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

In [ ]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {"number":i}
    producer.send(topic_name, value=message)
    print(f'Sent: {message}')
    time.sleep(0.05)
print(time.time()-t0)
producer.flush()
t1 = time.time()
print(f"Tool {(t1-t0)} senconds")

In [ ]:
0.6194324493408203 - 0.6194324493408203 

In [ ]:
import pandas as pd 

In [ ]:
df_green = pd.read_csv('green_tripdata_2019-10.csv')

In [ ]:
df_green_process = df_green[['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'tip_amount']]

In [ ]:
df_green_process

In [30]:
topic_name = 'green-trips'
t0 = time.time()
for row in df_green_process.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    print(row_dict)
    producer.send(topic = topic_name, value=row_dict)
    # time.sleep(2)
print(time.time()- t0)

{'lpep_pickup_datetime': '2019-10-01 00:26:02', 'lpep_dropoff_datetime': '2019-10-01 00:39:58', 'PULocationID': 112, 'DOLocationID': 196, 'passenger_count': 1.0, 'trip_distance': 5.88, 'tip_amount': 0.0}
{'lpep_pickup_datetime': '2019-10-01 00:18:11', 'lpep_dropoff_datetime': '2019-10-01 00:22:38', 'PULocationID': 43, 'DOLocationID': 263, 'passenger_count': 1.0, 'trip_distance': 0.8, 'tip_amount': 0.0}
{'lpep_pickup_datetime': '2019-10-01 00:09:31', 'lpep_dropoff_datetime': '2019-10-01 00:24:47', 'PULocationID': 255, 'DOLocationID': 228, 'passenger_count': 2.0, 'trip_distance': 7.5, 'tip_amount': 0.0}
{'lpep_pickup_datetime': '2019-10-01 00:37:40', 'lpep_dropoff_datetime': '2019-10-01 00:41:49', 'PULocationID': 181, 'DOLocationID': 181, 'passenger_count': 1.0, 'trip_distance': 0.9, 'tip_amount': 0.0}
{'lpep_pickup_datetime': '2019-10-01 00:08:13', 'lpep_dropoff_datetime': '2019-10-01 00:17:56', 'PULocationID': 97, 'DOLocationID': 188, 'passenger_count': 1.0, 'trip_distance': 2.52, 'tip

# CONSUME DATA PYSPARK

In [14]:
import pyspark 
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"
# SparkSession.setLevel('INFO')
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Green Trips Consumer") \
    .config('spark.jars.packages', kafka_jar_package) \
    .getOrCreate()

In [15]:
spark

In [16]:
lookup_static_df = spark.read.options(header=True).csv('D:/coding/project/project_study/Project_DE/05-Batch-processing/taxi_zone_lookup.csv')


In [29]:
lookup_static_df.groupBy('Zone', 'LocationID').count().groupBy('Zone', 'LocationID').max('count').show(20, False)

+-----------------------------+----------+----------+
|Zone                         |LocationID|max(count)|
+-----------------------------+----------+----------+
|East Harlem North            |74        |1         |
|Middle Village               |160       |1         |
|Long Island City/Queens Plaza|146       |1         |
|Morrisania/Melrose           |167       |1         |
|NA                           |265       |1         |
|East Tremont                 |78        |1         |
|Flushing                     |92        |1         |
|Meatpacking/West Village West|158       |1         |
|Greenpoint                   |112       |1         |
|Manhattan Beach              |150       |1         |
|Glen Oaks                    |101       |1         |
|Inwood Hill Park             |128       |1         |
|Lincoln Square West          |143       |1         |
|Queensboro Hill              |192       |1         |
|Central Harlem North         |42        |1         |
|Bayside                    

In [17]:
lookup_static_df.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [19]:
from pyspark.sql.functions import *

In [20]:
lookup_static_df.alias('t1').join(lookup_static_df.alias('t2'), col('t1.LocationID')==col('t2.LocationID'))

DataFrame[LocationID: string, Borough: string, Zone: string, service_zone: string, LocationID: string, Borough: string, Zone: string, service_zone: string]

In [ ]:
# sc = pyspark.SparkContext.getOrCreate()
# log4j = sc._jvm.org.apache.log4j
# log4j.LogManager.getRootLogger().setLevel(log4j.Level.DEBUG)

In [ ]:
topic_name = 'green-trips'
server = 'redpanda-1:9092'
server = 'localhost:9092'
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", server) \
    .option("subscribe", topic_name) \
    .option('startingOffsets', 'earliest') \
    .load()

In [ ]:
green_stream.printSchema()

In [ ]:
def  peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek)

In [ ]:
query.start()

In [ ]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [ ]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")